In [1]:
import pickle
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.nn.functional as F
from torch_geometric.nn import GATConv, SAGPooling,global_mean_pool , global_max_pool 
from sklearn.metrics import mean_absolute_error
from torch import optim

## fingat源码的关于编码的代码

In [17]:
class AttentionBlock(nn.Module):
    def __init__(self,time_step,dim):
        super(AttentionBlock, self).__init__()
        self.attention_matrix = nn.Linear(time_step, time_step)

    def forward(self, inputs):
        inputs_t = torch.transpose(inputs,2,1) # (batch_size, input_dim, time_step)
        attention_weight = self.attention_matrix(inputs_t)
        attention_probs = F.softmax(attention_weight,dim=-1)
        attention_probs = torch.transpose(attention_probs,2,1)
        attention_vec = torch.mul(attention_probs, inputs)
        attention_vec = torch.sum(attention_vec,dim=1)
        return attention_vec, attention_probs

class SequenceEncoder(nn.Module):
    def __init__(self,input_dim,time_step,hidden_dim):
        super(SequenceEncoder, self).__init__()
        self.encoder = nn.GRU(input_size=input_dim,hidden_size=hidden_dim,num_layers=1,batch_first=True)
        self.attention_block = AttentionBlock(time_step,hidden_dim) 
        self.dropout = nn.Dropout(0.2)
        self.dim = hidden_dim
    
    def forward(self,seq):
        '''
        inp : torch.tensor (batch,time_step,input_dim)
        '''
        seq_vector,_ = self.encoder(seq)
        seq_vector = self.dropout(seq_vector)
        attention_vec, attention_probs = self.attention_block(seq_vector)
        attention_vec = attention_vec.view(-1,1,self.dim) # prepare for concat
        return attention_vec, attention_probs

## 导入数据

In [3]:
with open('./datasets/sp500_data.pkl', "rb") as f:
    data = pickle.load(f)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [5]:
train_x = data['train']['x1'][:,:,:,1:]
train_x.shape

(1580, 480, 7, 29)

In [14]:
test_x = data['test']['x1'][:,:,:,1:]
test_x.shape

(390, 480, 7, 29)

In [6]:
train_y = data['train']['y_return ratio']
train_y.shape

(1580, 480)

In [16]:
test_y = data['test']['y_return ratio']
test_y.shape

(390, 480)

## 设立pytorch的dataloader

In [18]:
import torch
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam

class StockDataset(Dataset):
    def __init__(self, x, y):
        self.x = x
        self.y = y

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        return self.x[idx], self.y[idx]

def create_dataloader(x, y, batch_size):
    dataset = StockDataset(x, y)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)
    return dataloader

In [8]:
dataloader = create_dataloader(train_x, train_y, batch_size=16)

In [9]:
for batch_idx, (seq, target) in enumerate(dataloader):
    print(batch_idx)
    print(seq.shape)
    print(target.shape)
    print("=========================")

0
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
1
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
2
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
3
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
4
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
5
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
6
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
7
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
8
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
9
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
10
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
11
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
12
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
13
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
14
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
15
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
16
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
17
torch.Size([16, 480, 7, 29])
torch.Size([16, 480])
18
torch.Size([16, 480, 7, 29])
torch.

## 建立模型，设置优化器、损失，进行训练

In [19]:
# 建立
sequence_encoder = SequenceEncoder(input_dim=29, time_step=7, hidden_dim=64)

# 定义损失和优化
criterion = nn.MSELoss()  # 因为是回归问题，所以我们使用均方误差损失
optimizer = Adam(sequence_encoder.parameters(), lr=0.001)

In [21]:
# 5. 训练模型
def train_model(model, dataloader, epochs):
    min_loss = float('inf')  # 初始最小损失设为正无穷大
    model = model.to(device)
    model.train()
    for epoch in range(epochs):
        for batch_idx, (seq, target) in enumerate(dataloader):
            seq = seq.to(device).float()
            target = target.to(device).float()
            optimizer.zero_grad()
            seq = seq.view(-1, 7, 29)  # Reshape to (batch_size * num_stocks, time_step, input_dim)
#             print(seq.shape)  #  [5984, 7, 30]
            target = target.view(-1,1)  # Flatten target to have shape (batch_size * num_stocks)
#             print(target.shape)  #  [5984, 1]
            attention_vec, attention_probs = model(seq)
            attention_vec = attention_vec.squeeze()  # Remove extra dimensions to match target shape
            loss = criterion(attention_vec, target)
            loss.backward()
            optimizer.step()
            if batch_idx % 20 == 0:
                print(f'Epoch {epoch}, Batch {batch_idx}, Loss {loss.item()}')
        if loss.item() < min_loss:  # 当前损失小于记录的最小损失就保存
            min_loss = loss.item() 
            torch.save(model, './output/model/2023-11-28/sequence_encoder.pkl') 
            print('save!')

# 开始训练
train_model(sequence_encoder, dataloader, epochs=50)

D:\_Application\anaconda3\envs\Pytorch\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([7680, 1])) that is different to the input size (torch.Size([7680, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 0, Batch 0, Loss 0.0099116126075387
Epoch 0, Batch 20, Loss 0.0043643685057759285
Epoch 0, Batch 40, Loss 0.0029200019780546427
Epoch 0, Batch 60, Loss 0.001389387296512723
Epoch 0, Batch 80, Loss 0.0008254718268290162


D:\_Application\anaconda3\envs\Pytorch\lib\site-packages\torch\nn\modules\loss.py:535: UserWarning: Using a target size (torch.Size([5760, 1])) that is different to the input size (torch.Size([5760, 64])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


save!
Epoch 1, Batch 0, Loss 0.0009037769632413983
Epoch 1, Batch 20, Loss 0.0006938229780644178
Epoch 1, Batch 40, Loss 0.0003547155938576907
Epoch 1, Batch 60, Loss 0.00046185782412067056
Epoch 1, Batch 80, Loss 0.00030233271536417305
save!
Epoch 2, Batch 0, Loss 0.0004561389214359224
Epoch 2, Batch 20, Loss 0.0003709917073138058
Epoch 2, Batch 40, Loss 0.0002968697517644614
Epoch 2, Batch 60, Loss 0.0003529299283400178
Epoch 2, Batch 80, Loss 0.0002861999091692269
Epoch 3, Batch 0, Loss 0.00045825602137483656
Epoch 3, Batch 20, Loss 0.00035936152562499046
Epoch 3, Batch 40, Loss 0.00032564232242293656
Epoch 3, Batch 60, Loss 0.0005724569200538099
Epoch 3, Batch 80, Loss 0.0003526262880768627
Epoch 4, Batch 0, Loss 0.0003190336574334651
Epoch 4, Batch 20, Loss 0.0003191427676938474
Epoch 4, Batch 40, Loss 0.00027990969829261303
Epoch 4, Batch 60, Loss 0.0005514786462299526
Epoch 4, Batch 80, Loss 0.00036044197622686625
Epoch 5, Batch 0, Loss 0.0003257881908211857
Epoch 5, Batch 20, L

Epoch 36, Batch 0, Loss 0.00044567036093212664
Epoch 36, Batch 20, Loss 0.0004202554700896144
Epoch 36, Batch 40, Loss 0.00040101970080286264
Epoch 36, Batch 60, Loss 0.00034417095594108105
Epoch 36, Batch 80, Loss 0.0005037141381762922
Epoch 37, Batch 0, Loss 0.0003154866863042116
Epoch 37, Batch 20, Loss 0.00042454441427253187
Epoch 37, Batch 40, Loss 0.00047529165749438107
Epoch 37, Batch 60, Loss 0.00041758466977626085
Epoch 37, Batch 80, Loss 0.00034215234336443245
Epoch 38, Batch 0, Loss 0.0003547605301719159
Epoch 38, Batch 20, Loss 0.00042689728434197605
Epoch 38, Batch 40, Loss 0.0003468747599981725
Epoch 38, Batch 60, Loss 0.00033570267260074615
Epoch 38, Batch 80, Loss 0.00027446908643469214
Epoch 39, Batch 0, Loss 0.00028230255702510476
Epoch 39, Batch 20, Loss 0.0005897572846151888
Epoch 39, Batch 40, Loss 0.00036200491013005376
Epoch 39, Batch 60, Loss 0.000549161050003022
Epoch 39, Batch 80, Loss 0.0003109727695118636
Epoch 40, Batch 0, Loss 0.0002779813075903803
Epoch 4

## 测试数据，加载node和edge

In [23]:
test_x.shape

(390, 480, 7, 29)

In [22]:
new_week_data = test_x

# 评估模型
sequence_encoder.eval()

# 不需要计算梯度
with torch.no_grad():
    input = torch.Tensor(new_week_data).view(-1, 7, 30).to(device)
    print(input.size())
    attention_vecs = sequence_encoder(input)

torch.Size([187200, 7, 29])


OutOfMemoryError: CUDA out of memory. Tried to allocate 5.10 GiB. GPU 0 has a total capacty of 6.00 GiB of which 3.13 GiB is free. Of the allocated memory 851.01 MiB is allocated by PyTorch, and 72.99 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [69]:
attention_vecs.size()

torch.Size([374, 1, 64])

In [70]:
inner_edge = np.array(np.load("/openbayes/input/input0/inner_edge.npy"))
inner_edge

array([[  0,   0],
       [  0,   5],
       [  0,  18],
       ...,
       [337, 337],
       [337, 349],
       [349, 349]], dtype=int32)

In [71]:
inner_edge = torch.tensor(inner_edge.T, dtype=torch.int64).to(device)
inner_edge

tensor([[  0,   0,   0,  ..., 337, 337, 349],
        [  0,   5,  18,  ..., 337, 349, 349]], device='cuda:0')

In [72]:
inner_edge.size()

torch.Size([2, 6346])

## 一周内、不同股票间的gat代码

In [73]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv

class GATModel(torch.nn.Module):
    def __init__(self, num_node_features, num_hidden_units, num_classes):
        super(GATModel, self).__init__()
        self.conv1 = GATConv(num_node_features, num_hidden_units, heads=1, dropout=0.6)
        self.conv2 = GATConv(num_hidden_units, num_classes, heads=1, concat=False, dropout=0.6)

    def forward(self, x, edge_index):
        # 输入层
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))

        # 输出层
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [74]:
# node和edge
nodes = attention_vecs.squeeze(1)  # 假设这是你的节点特征
print(nodes.size())
edges = inner_edge  # 假设这是你的边索引
print(edges.size())

# 创建模型
num_node_features = 64
num_hidden_units = 8
num_classes = 8
model = GATModel(num_node_features, num_hidden_units, num_classes).to(device)

# 前向传播
model.train()
out = model(nodes, edges)


torch.Size([374, 64])
torch.Size([2, 6346])


In [75]:
out.size()

torch.Size([374, 8])